# Q4:b)Implement Gradient Descent algorithm in the following manner:  (Give it a try)
a. First take a dataset with single independent variable and find the best fit using Simple Linear Regression. 

b. Using the values of coefficients given in step a, apply gradient descent to minimize loss function and then make prediction again. 

c. Compare the best fit line of case a and b. 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
seed = 23

In [ ]:
df = pd.read_csv("experience_salary.csv")

In [ ]:
df.head()

,YearsExperience,Salary
0,1.1,39343.0
1,1.3,46205.0
2,1.5,37731.0
3,2.0,43525.0
4,2.2,39891.0


In [ ]:
df.isna().sum()

YearsExperience    0
Salary             0
dtype: int64

In [ ]:
df.describe()

,YearsExperience,Salary
count,30.000000,30.000000
mean,5.313333,76003.000000
std,2.837888,27414.429785
min,1.100000,37731.000000
25%,3.200000,56720.750000
50%,4.700000,65237.000000
75%,7.700000,100544.750000
max,10.500000,122391.000000


In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=seed)

## Linear regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_pred)

30404597.28731416

In [ ]:
squared_error = mean_squared_error(y_test,y_pred)*y_test.shape[0]
squared_error

182427583.72388497

In [ ]:
y_test

array([66029., 91738., 63218., 67938., 98273., 39891.])

In [ ]:
y_pred

array([74501.41002938, 90279.63442954, 63363.83986457, 72645.14833525,
       93064.02697075, 47585.61546441])

In [ ]:
lr.coef_

array([9281.30847068])

##Gradient Descent
<font size=4></br>Gradient descent is very helpful in finding the minima of the loss function.
</br>step 1: take derivative of loss function
</br>step 2: start off with random values (generally constant is 0 while a factor is given a value of 1)
</br>step 3: now put these values into the loss function's derivative
</br>step 4: step size = result from step 3* learning rate(shouldn't be big enough that it jumps over the minima)
</br>step 5: new value for coefficients = old value-step size
</br>step 6: repeat above steps until step size becomes very small or max iteration is reached
</font>

In [ ]:
X = df.iloc[:,:1].values
y = df.iloc[:,-1:].values
X_train2,X_test2,y_train2,y_test2 = train_test_split(X,y,random_state=seed,test_size=0.2)
data = np.concatenate((X_train2,y_train2),axis=1)

In [ ]:
from sympy import symbols,lambdify
x,y,a,b = symbols('x y a b')
lsf = (y-(a*x+b))**2
lsf_a = lsf.diff(a)
lsf_b = lsf.diff(b)
lsf_b_f = lambdify((x,y,a,b),lsf_b)
lsf_a_f = lambdify((x,y,a,b),lsf_a)
def se_calc(f,m,n,data):
    return sum([f(i,j,m,n) for i,j in data])

In [ ]:
def gradient_descent(m,n,data,learning_rate,k,min_step):
    for i in range(k):
        step_size_a = se_calc(lsf_a_f,m,n,data)*learning_rate
        step_size_b = se_calc(lsf_b_f,m,n,data)*learning_rate
        if abs(step_size_a)<min_step and abs(step_size_b)<min_step:
            return m,n
        if abs(step_size_b)>=min_step:
            n-=step_size_b
        if abs(step_size_a)>=min_step:
            m-= step_size_a
    return m,n

In [ ]:
m,n = gradient_descent(1,0,data,0.00001,20000,0.00005)

In [ ]:
test_data = np.concatenate((X_test2,y_test2),axis=1)

In [ ]:
print(m,n)

9671.122623058573 24430.758032391866


In [ ]:
lsf_f = lambdify((x,y,a,b),lsf)
se_calc(lsf_f,m,n,test_data)

138891793.39133903

In [ ]:
y_pred2 = list()
for i in range(X_test.shape[0]):
    y_pred2.append(m*X_test[i][0]+n)
y_pred2 = np.array(y_pred2)

In [ ]:
mean_squared_error(y_test,y_pred2)*y_pred2.shape[0]

138891793.39133903

In [ ]:
t = np.concatenate((np.array(list(map(lambda x: [x],list(y_pred)))),y_test2,(np.array(list(map(lambda x: [x],list(y_pred2)))))),axis=1)

In [ ]:
t

array([[74501.41002938, 66029.        , 73753.48340999],
       [90279.63442954, 91738.        , 90194.39186919],
       [63363.83986457, 63218.        , 62148.13626232],
       [72645.14833525, 67938.        , 71819.25888538],
       [93064.02697075, 98273.        , 93095.72865611],
       [47585.61546441, 39891.        , 45707.22780312]])

In [ ]:
gde = mean_squared_error(y_test,y_pred2)*y_pred2.shape[0]
lre = mean_squared_error(y_test,y_pred)*y_pred.shape[0]
print(f'gradient descent: {gde}\nlinear rigression: {lre}\ndifference(reg-grad):{lre-gde}')

gradient descent: 138891793.39133903
linear rigression: 182427583.72388497
difference(reg-grad):43535790.332545936
